## debug the transformation problem

```bash
File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/engine/trainer.py", line 118, in do_train
    for iteration, (images, targets, _) in enumerate(data_loader, start_iter):
  File "/home/r7user3/anaconda2/envs/maskrcnn/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 615, in __next__
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "/home/r7user3/anaconda2/envs/maskrcnn/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 615, in <listcomp>
    batch = self.collate_fn([self.dataset[i] for i in indices])
  File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/data/datasets/coco.py", line 61, in __getitem__
    img, target = self.transforms(img, target)
  File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/data/transforms/transforms.py", line 19, in __call__
    image, target = t(image, target)
  File "/home/r7user3/ZhangHT/github/maskrcnn-benchmark/maskrcnn_benchmark/data/transforms/transforms.py", line 95, in __call__
    target=F.rotate(target,angle)
  File "/home/r7user3/anaconda2/envs/maskrcnn/lib/python3.7/site-packages/torchvision/transforms/functional.py", line 593, in rotate
    raise TypeError('img should be PIL Image. Got {}'.format(type(img)))
TypeError: img should be PIL Image. Got <class 'maskrcnn_benchmark.structures.bounding_box.BoxList'>

```

### train_net.py

In [3]:
from maskrcnn_benchmark.config import cfg
import os
import torch
from maskrcnn_benchmark.utils.comm import get_world_size
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.solver import make_lr_scheduler
from maskrcnn_benchmark.solver import make_optimizer

In [4]:
def reduce_loss_dict(loss_dict):
    """
    Reduce the loss dictionary from all processes so that process with rank
    0 has the averaged results. Returns a dict with the same fields as
    loss_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return loss_dict
    with torch.no_grad():
        loss_names = []
        all_losses = []
        for k in sorted(loss_dict.keys()):
            loss_names.append(k)
            all_losses.append(loss_dict[k])
        all_losses = torch.stack(all_losses, dim=0)
        dist.reduce(all_losses, dst=0)
        if dist.get_rank() == 0:
            # only main process gets accumulated, so only divide by
            # world_size in this case
            all_losses /= world_size
        reduced_losses = {k: v for k, v in zip(loss_names, all_losses)}
    return reduced_losses


In [5]:
# simulate the true working dir
os.chdir('/home/r7user3/ZhangHT/github/maskrcnn-benchmark/')

In [6]:
cfg.merge_from_file('./configs/e2e_mask_rcnn_R_50_FPN_1x.yaml')
cfg.freeze()

In [7]:
distributed=False
start_iter=0

In [8]:
data_loader = make_data_loader(
        cfg,
        is_train=True,
        is_distributed=distributed,
        start_iter=start_iter,
    )

When using more than one image per GPU you may encounter an out-of-memory (OOM) error if your GPU does not have sufficient memory. If this happens, you can reduce SOLVER.IMS_PER_BATCH (for training) or TEST.IMS_PER_BATCH (for inference). For training, you must also adjust the learning rate and schedule length according to the linear scaling rule. See for example: https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14


loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


### trainer.py

This Problem is caused by **too large** learning rate

In [9]:
model=build_detection_model(cfg)
device = torch.device(cfg.MODEL.DEVICE)
device

device(type='cuda')

In [10]:
model.to(device)
optimizer = make_optimizer(cfg, model)
scheduler = make_lr_scheduler(cfg, optimizer)

In [27]:
max_iter = 100
start_iter = 0 # 0 at start

model.train() # begin training

######################################################################################################################
# Note: once begin training, rank of GPU will change iteratively! So the rank of GPU is 0 above this comment.
#       Moreover, iteration is counted following rank 0 GPU. In other words, all checkpoint work on cuda:0.
######################################################################################################################
for iteration, (images, targets,_) in enumerate(data_loader,0):
    

    iteration = iteration + 1
    print('iteration:',iteration)

    scheduler.step()
    print('cp1')

    images = images.to(device)
    targets = [target.to(device) for target in targets]
    print('cp2')
    loss_dict = model(images, targets)

    losses = sum(loss for loss in loss_dict.values())

    # reduce losses over all GPUs for logging purposes
    loss_dict_reduced = reduce_loss_dict(loss_dict)
    losses_reduced = sum(loss for loss in loss_dict_reduced.values())

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

    if iteration % 20 == 0 or iteration == max_iter:
        print('Loss',losses.item())

iteration: 1
cp1


RuntimeError: CUDA error: device-side assert triggered

## multi train

Learning rate is too large, suggest to be 0.0025.
```bash
CUDA_VISIBLE_DEVICES=2,3 python -m torch.distributed.launch --nproc_per_node=2 --master_addr 127.0.0.2 --master_port 29501 tools/train_net.py --config-file configs/e2e_mask_rcnn_R_50_FPN_1x.yaml SOLVER.IMS_PER_BATCH 4 SOLVER.BASE_LR 0.005 SOLVER.MAX_ITER 30000 SOLVER.STEPS "(15000, 20000)" TEST.IMS_PER_BATCH 2
```

```bash
python tools/train_net.py --config-file "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" SOLVER.BASE_LR 0.0025 SOLVER.MAX_ITER 60000 SOLVER.STEPS "(30000, 40000)" TEST.IMS_PER_BATCH 1
```

```bash
python tools/train_net.py --config-file "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" SOLVER.BASE_LR 0.0025 TEST.IMS_PER_BATCH 1
```

```bash
CUDA_VISIBLE_DEVICES=2,3 python -m torch.distributed.launch --nproc_per_node=2 --master_addr 127.0.0.2 --master_port 29501 tools/train_net.py --config-file configs/e2e_mask_rcnn_R_50_FPN_1x.yaml SOLVER.BASE_LR 0.0025 SOLVER.IMS_PER_BATCH 4 TEST.IMS_PER_BATCH 2
```

## validation

In [51]:
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.data import make_data_loader
from maskrcnn_benchmark.engine.inference import inference
from maskrcnn_benchmark.modeling.detector import build_detection_model
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer

from maskrcnn_benchmark.utils.validation import EvalMetric
from maskrcnn_benchmark.modeling.roi_heads.mask_head.inference import Masker

from maskrcnn_benchmark.config.paths_catalog import DatasetCatalog
from pycocotools.coco import COCO
import numpy as np

import matplotlib.pyplot as plt

import os
import torch

In [5]:
os.chdir('/home/r7user3/ZhangHT/github/maskrcnn-benchmark/')

In [38]:
cfg.merge_from_file('./configs/e2e_mask_rcnn_R_50_FPN_1x.yaml')
cfg.merge_from_list(['MODEL.WEIGHT',"./logs/rectal_benchmark/model_final.pth",'TEST.IMS_PER_BATCH',1])
cfg.freeze()

In [39]:
cfg.MODEL.WEIGHT

'./logs/rectal_benchmark/model_final.pth'

In [41]:
iou_types = ("bbox",)
if cfg.MODEL.MASK_ON:
    iou_types = iou_types + ("segm",)
    dataset_name = cfg.DATASETS.VAL

In [40]:
model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)
data_loader_val = make_data_loader(cfg, is_train=False, is_val=True)[0]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [52]:
output_dir = './'
checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
_ = checkpointer.load(cfg.MODEL.WEIGHT)

In [53]:
predictions=inference(
                        model,
                        data_loader_val,
                        dataset_name=dataset_name,
                        iou_types=iou_types,
                        box_only=cfg.MODEL.RPN_ONLY,
                        device=cfg.MODEL.DEVICE,
                        expected_results=cfg.TEST.EXPECTED_RESULTS,
                        expected_results_sigma_tol=cfg.TEST.EXPECTED_RESULTS_SIGMA_TOL,
                        only_predictions=True
                    )

100%|██████████| 84/84 [00:07<00:00, 10.97it/s]


In [54]:
_,ann_file=DatasetCatalog.DATASETS[cfg.DATASETS.VAL[0]].values()
data_dir=DatasetCatalog.DATA_DIR
annFile=data_dir+'/'+ann_file

coco=COCO(annFile)
imgIds=coco.getImgIds()
imgsInfo=coco.loadImgs(imgIds)
whs=[(img['width'],img['height']) for img in imgsInfo]

print('Loading the masks...')
masksgt=[]
for i,imgId in enumerate(imgIds):
    annIds=coco.getAnnIds(imgIds=imgId)
    anns=coco.loadAnns(annIds)
    maskgt=[]
    for ann in anns:
        maskgt.append(coco.annToMask(ann))
    masksgt.append((np.sum(maskgt,0)>0).astype(np.uint8))
print('Loading Complete!')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading the masks...
Loading Complete!


In [88]:
masker=Masker(cfg.EVAL_THRESHOLD)
print('Loading the predictions...')
masksdt=[]
for i,prediction in enumerate(predictions):
    prediction=prediction.resize(whs[i])
    if not len(prediction):
        # if num of box is 0, there is no mask (2019/01/16)
        maskdt=np.zeros(tuple(whs[i][::-1]),dtype=np.uint8)
    else:
        maskdt=prediction.get_field('mask')
        if list(maskdt.shape[-2:]) != list(whs[i][::-1]):
            maskdt = masker(maskdt.expand(1, -1, -1, -1, -1), prediction)
            maskdt = maskdt[0]
        maskdt=maskdt.numpy().sum((0,1))
        maskdt=(maskdt>0).astype(np.uint8)
    masksdt.append(maskdt)
print('Loading Complete!')

Loading the predictions...
Loading Complete!


In [89]:
mean_dice=EvalMetric(masksgt,masksdt).mean_dice

In [90]:
mean_dice

0.5960018956931298